In [1]:
suppressMessages(library(rwwa))
png_res <- 240

In [2]:
mdl_args <- list("hdwi_chile2026" = list(dist = "gev", type = "shiftscale", lower = F, covnm = c("gmst")),
                 "hdwi_patagonia" = list(dist = "gev", type = "shift", lower = F, covnm = c("gmst")),
                 "pr-ndj_chile2026" = list(dist = "norm_logt", type = "shift", lower = T, covnm = c("gmst", "samdet", "nino34")),
                 "pr-ndj_patagonia" = list(dist = "norm_logt", type = "shift", lower = T, covnm = c("gmst", "samdet", "nino34")),
                 "pet-ndj_chile2026" = list(dist = "norm", type = "shift", lower = F, covnm = c("gmst", "samdet", "nino34")),
                 "pet-ndj_patagonia" = list(dist = "norm", type = "shift", lower = F, covnm = c("gmst", "samdet", "nino34")),
                 "epr-ndj_chile2026" = list(dist = "norm", type = "shift", lower = T, covnm = c("gmst", "samdet", "nino34")),
                 "epr-ndj_patagonia" = list(dist = "norm", type = "shift", lower = T, covnm = c("gmst", "samdet", "nino34")))


cov_df <- merge(merge(read.table("ts-obs/gmst-smoothed.dat", col.names = c("year", "gmst")),
                      load_ts("ts-obs/sam-fires_nino34det-ndj.dat", col.names = c("year", "nino34"))),
                load_ts("ts-obs/sam-fires_sam-dec.dat", col.names = c("year", "sam")))
cov_df$gmst <- cov_df$gmst - cov_df$gmst[cov_df$year == 2026]
cov_df$samdet <- resid(lm(sam ~ gmst, cov_df))


cov_2026 <- cov_df[cov_df$year == 2026,c("gmst", "sam", "samdet", "nino34"),drop = F]
cov_cf <- rbind("pi" = cov_2026 - c(1.3, 0, 0, 0),
                "samneutral" = c(cov_2026$gmst, 0, 0, cov_2026$nino34),
                "ensoneutral" = c(cov_2026$gmst, cov_2026$sam, cov_2026$samdet, 0),
                "bothneutral" = c(cov_2026$gmst, 0, 0, 0))

cov_list <- list("gmst-only" = "gmst", "gmst+nino" = c("gmst", "nino34"), "gmst+sam" = c("gmst", "sam"), 
                 "gmst+nino+sam" = c("gmst", "sam", "nino34")
                 ,"gmst+samdet" = c("gmst", "samdet"),"gmst+nino+samdet" = c("gmst", "samdet", "nino34")
                )

# Tables of results

In [3]:
varnm <- "hdwi"
rnm <- "chile2026"

res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")
res <- res[c('gmst-only','gmst+nino','gmst+samdet','gmst+nino+samdet'),]

In [4]:
cols <- c("event", "dI.rel.pi", "dI.rel.ensoneutral", "dI.rel.samneutral", "return.period", "aic")
sapply(cols, function(cnm) {
    apply(signif(res[,grepl(cnm, colnames(res)),drop = F], 3), 1, function(r) {
        paste0(r[1], " (",r[2], ", ", r[3], ")")
    })
})

,event,dI.rel.pi,dI.rel.ensoneutral,dI.rel.samneutral,return.period,aic
gmst-only,"20.4 (20.4, 20.4)","45.5 (11.9, 66)","0 (0, 0)","0 (0, 0)","4.19 (2.27, 18.5)","229 (203, 246)"
gmst+nino,"20.4 (20.4, 20.4)","34.5 (-53, 66.5)","9.73 (-6.17, 32.2)","0 (0, 0)","4.38 (2.2, 17.2)","230 (198, 243)"
gmst+samdet,"20.4 (20.4, 20.4)","45.4 (-19, 73.4)","0 (0, 0)","2.75 (-9.17, 21.9)","3.95 (2.11, 20.6)","232 (200, 247)"
gmst+nino+samdet,"20.4 (20.4, 20.4)","20.9 (-97.2, 74.7)","18.9 (-8.81, 67.1)","10 (-7.04, 35.3)","3.47 (1.76, 12.9)","230 (175, 243)"


In [59]:
cols <- c("event", "dI.abs.pi", "dI.abs.ensoneutral", "dI.abs.samneutral", "return.period",  "aic")
sapply(cols, function(cnm) {
    apply(signif(res[,grepl(cnm, colnames(res)),drop = F], 3), 1, function(r) {
        paste0(r[1], " (",r[2], ", ", r[3], ")")
    })
})

,event,dI.abs.pi,dI.abs.ensoneutral,dI.abs.samneutral,return.period,aic
gmst-only,"-1.74 (-1.74, -1.74)","-1.45 (-2.74, -0.2)","0 (0, 0)","0 (0, 0)","4.57 (2.37, 16.1)","154 (132, 168)"
gmst+nino,"-1.74 (-1.74, -1.74)","-1.23 (-2.68, 0.246)","-0.334 (-0.904, 0.112)","0 (0, 0)","4.25 (2.12, 18.2)","154 (129, 166)"
gmst+samdet,"-1.74 (-1.74, -1.74)","-1.45 (-2.67, -0.147)","0 (0, 0)","0.642 (0.213, 1.13)","13.2 (4.75, 202)","148 (120, 164)"
gmst+nino+samdet,"-1.74 (-1.74, -1.74)","-1.34 (-2.62, 0.121)","-0.174 (-0.777, 0.327)","0.589 (0.0739, 1.1)","11.4 (3.97, 229)","150 (119, 163)"


In [56]:
cols <- c("return.period", "PR.pi", "PR.ensoneutral", "PR.samneutral", "aic")
sapply(cols, function(cnm) {
    apply(signif(res[,grepl(cnm, colnames(res)),drop = F], 3), 1, function(r) {
        paste0(r[1], " (",r[2], ", ", r[3], ")")
    })
})

,return.period,PR.pi,PR.ensoneutral,PR.samneutral,aic
gmst-only,"1.92 (1.29, 3.97)","24 (4.77, Inf)","1 (1, 1)","1 (1, 1)","110 (89.1, 122)"
gmst+nino,"1.93 (1.25, 4.18)","25.9 (4.31, Inf)","0.963 (0.684, 1.46)","1 (1, 1)","112 (89.1, 123)"
gmst+samdet,"2.6 (1.47, 7.11)","1170 (4.79, Inf)","1 (1, 1)","0.697 (0.358, 1.07)","110 (86.2, 116)"
gmst+nino+samdet,"3.6 (1.56, 10.4)","Inf (3.94, Inf)","0.74 (0.406, 1.56)","0.516 (0.219, 1.08)","111 (84.2, 116)"


# Figures for obs discussion

In [43]:
df <- merge(cov_df, load_ts(paste0("ts-obs/sam-fires_",varnm,"_",rnm,"_era5land.dat"), col.names = c("year", "x")))
if (varnm %in% c("pr-ndj")) df$x <- df$x * 92

In [44]:
mdl <- do.call(fit_ns, append(mdl_args[[paste0(varnm,"_",rnm)]], list("data" = df, "varnm" = "x")))

## Trends vs covariates

In [45]:
png(paste0("fig/covtrend_",varnm,"_",rnm,".png"), h = png_res, w = png_res * 3); {
    prep_window(c(1,3), cex.axis = 1.25, mar = c(4,2,3,1), oma = c(0,3,0,0))

    plot_covtrend(mdl, "gmst", ylab = "", xlab = "", legend_pos = NA)
    mtext("NDJ precipitation (mm)", side = 2, line = 3.25)
    mtext("GMST anomaly wrt 2026", side = 1, line = 2.75)
    mtext("(a)", side = 3, line = 0.75, adj = 0, font = 2)
    
    plot_covtrend(mdl, "nino34", ylab = "", xlab = "", legend_pos = NA)
    mtext("Detrended Nino3.4 index", side = 1, line = 2.75)
    mtext("(b)", side = 3, line = 0.75, adj = 0, font = 2)
    
    plot_covtrend(mdl, "samdet", ylab = "", xlab = "", legend_pos = NA)
    mtext("Detrended SAM index", side = 1, line = 2.75)
    mtext("(c)", side = 3, line = 0.75, adj = 0, font = 2)
}; dev.off()

agg_record_300909875 
                   2

## Trends & GMST return level plot

In [46]:
png(paste0("fig/trend-and-rl_",varnm,"_",rnm,".png"), h = png_res*1.5, w = png_res*1.5 * 3); {
    
    prep_window(c(1,2), cex.axis = 1.25, cex.lab = 1.25, mar = c(4,2,2.5,1), oma = c(0,3,0,0))

    plot_trend(mdl, ylab = "", xlab = "", legend_pos = NA)
    ylab_text = switch(varnm,
                       "pr-ndj" = "NDJ precipitation (mm)",
                       "hdwi" = "HDWI (kPa m/s)",
                       "x")
    mtext(ylab_text, side = 2, line = 3.25, cex = 1.25)
    
    mtext("Year", side = 1, line = 2.75, cex = 1.5)
    mtext("(a)", side = 3, line = 0.75, adj = 0, font = 2)

    plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["pi",,drop = F], ylab = "", xlab = "",
                      model_desc = F, legend_labels = c("2026", "1.3°C cooler climate"), legend_pos = "bottomright")
    mtext("Return period (years)", side = 1, line = 2.75, cex = 1.5)
    mtext("(b)", side = 3, line = 0.75, adj = 0, font = 2)

}; dev.off()

agg_record_665579472 
                   2

## Return level plots

In [ ]:
# png(paste0("fig/rlplots_",varnm,"_",rnm,".png"), h = png_res, w = png_res * 3); {
#     prep_window(c(1,3), cex.axis = 1.25, mar = c(4,2,3,1), oma = c(0,3,0,0))

#     plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["pi",,drop = F], ylab = "", legend_pos = NA, nsamp = 5)
#     mtext("NDJ precipitation (mm)", side = 2, line = 3.25)
#     mtext("(a)", side = 3, line = 0.75, adj = 0, font = 2)

#     plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["ensoneutral",,drop = F], ylab = "", legend_pos = NA, nsamp = 5)
#     mtext("(b)", side = 3, line = 0.75, adj = 0, font = 2)
    
#     plot_returnlevels(mdl, cov_f = cov_2026, cov_cf = cov_cf["samneutral",,drop = F], ylab = "", legend_pos = NA, nsamp = 5)
#     mtext("(c)", side = 3, line = 0.75, adj = 0, font = 2)
    
# }; dev.off()

# Model evaluation tables

In [139]:
varnm <- "pr-ndj"
rnm <- "patagonia"

obs_res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")[c("gmst-only"),,drop = F]
mdl_res <- read.csv(paste0("res-cordex_",varnm,"_gmst-only_",rnm,"_with-eval.csv"), row.names = "X.1")

In [140]:
obs_df <- data.frame("Event" = signif(obs_res$event.magnitude_est,3),
                     "Seasonal" = "",
                     "Spatial" = "", 
                     "Scale parameter" = apply(signif(obs_res[,grepl("sigma0", colnames(obs_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Shape parameter" = apply(signif(obs_res[,grepl("shape", colnames(obs_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Overall" = "",
                     "Keep" = "")
rownames(obs_df) <- c("ERA5-Land")

mdl_df <- data.frame("Event" = signif(mdl_res$rp_value,3),
                     "Seasonal" = mdl_res$seasonal,
                     "Spatial" = mdl_res$spatial, 
                     "Scale parameter" = apply(signif(mdl_res[,grepl("sigma0", colnames(mdl_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Shape parameter" = apply(signif(mdl_res[,grepl("shape", colnames(mdl_res)),drop = F], 3), 1, function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")),
                     "Overall" = mdl_res$cat,
                     "Keep" = mdl_res$include)
rownames(mdl_df) <- gsub("i1p1_", "i1p1 ",rownames(mdl_df))

spacer_df <- data.frame(sapply(colnames(obs_df), function(cnm) rep("",2)), row.names = c("", "CORDEX"))

# Model results

In [258]:
varnm <- "pr-ndj"
rnm <- "chile2026"

obs_res <- read.csv(paste0("res-obs_",varnm,"_",rnm,".csv"), row.names = "X")[c("gmst-only"),,drop = F]

mdl_res <- read.csv(paste0("res-cordex_",varnm,"_gmst-only_",rnm,"_with-eval.csv"), row.names = "X.1")
print(nrow(mdl_res))
mdl_res <- mdl_res[mdl_res$include == "Y",]
print(nrow(mdl_res))

[1] 18
[1] 8


In [250]:
cols <- switch(varnm, "hdwi" = c("dI.abs", "PR"), "pr-ndj" = c("dI.rel", "PR"))

mdl_df <- data.frame(cbind(sapply(cols, function(cnm) apply(signif(mdl_res[,grepl(paste0("attr_",cnm), colnames(mdl_res)),drop = F], 3), 1, 
                                                           function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))),
                     cbind(sapply(cols, function(cnm) apply(signif(mdl_res[,grepl(paste0("proj_",cnm), colnames(mdl_res)),drop = F], 3), 1, 
                                                           function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))))
rownames(mdl_df) <- gsub("i1p1_", "i1p1 ",rownames(mdl_df))


In [244]:
obs_df <- data.frame(t(sapply(cols, function(cnm) apply(signif(obs_res[,grepl(cnm, colnames(obs_res)),drop = F], 3), 1, 
                                                        function(r) paste0(r[1], " (",r[2], ", ", r[3], ")")))))
colnames(obs_df) <- colnames(mdl_df)[1:length(colnames(obs_df))]
rownames(obs_df) <- c("ERA5-Land")


In [245]:
df <- data.frame(rbind.fill(obs_df, mdl_df), row.names = c(rownames(obs_df), rownames(mdl_df)))

In [246]:
cat(rnm,varnm)
df[,c(1,3,2,4)]

patagonia pr-ndj

,dI.rel,dI.rel.1,PR,PR.1
,<chr>,<chr>,<chr>,<chr>
ERA5-Land,"-35 (-58.5, -3.46)",NA,"9.08 (1.22, 104)",NA
SAM-22_HadGEM2-ES_r1i1p1 RegCM4-7,"-15.3 (-31.1, 3.01)","-8.8 (-16.2, -1.46)","2.6 (0.863, 17.3)","1.45 (1.07, 1.93)"
SAM-22_HadGEM2-ES_r1i1p1 REMO2015,"-14.6 (-30, 3.18)","-11.5 (-19, -4.25)","2.49 (0.853, 14.6)","1.64 (1.22, 2.17)"
SAM-22_NorESM1-M_r1i1p1 RegCM4-7,"-6.36 (-27.3, 17.6)","-15.4 (-26, -5.09)","1.53 (0.44, 18.1)","1.92 (1.29, 2.68)"
SAM-22_NorESM1-M_r1i1p1 REMO2015,"-8.62 (-31.3, 16.6)","-16 (-27, -5.73)","1.79 (0.449, 20.4)","1.96 (1.34, 2.75)"
SAM-44_HadGEM2-ES_r1i1p1 RegCM4-3,"-8.72 (-28, 14.6)","-3.47 (-11.6, 3.44)","1.56 (0.576, 8.32)","1.17 (0.841, 1.64)"
